# Horário Escolar

Importando bibliotecas:

In [177]:
import numpy as np
import random as rd
import pandas as pd
from typing import List
from collections import defaultdict

In [178]:
def fit(mClasses:np.ndarray, mUsed:np.ndarray, mExcept:np.ndarray, sequence:int) -> List[tuple]:
    for x, y in np.ndindex(mClasses.shape):
        pos = y - (sequence - 1)
        if pos < 0:
            pos = 0
        if y >= (sequence -1):
            testRow = np.full((1, sequence), x)[0]
            if not np.any(mClasses[testRow, list(range(pos,y+1))]):
                if not np.any(mUsed[testRow, list(range(pos,y+1))]):
                    if not np.any(mExcept[testRow, list(range(pos,y+1))]):
                        mClasses[x, pos:pos + sequence] = 1
                        mUsed[x, pos:pos + sequence] = 1
                        return (x, pos)
    return False

In [184]:
grades = {
    '3-medio-basico'    :   {
        'arte': 1, 'biologia': 3, 'ed-fisica': 1, 'filosofia': 1, 'física': 3, 'geografia': 2, 'historia': 2,
        'ingles': 2, 'portugues': 4, 'matematica': 4, 'quimica': 4, 'sociologia': 1, 'redacao': 1
    }
}
turmas = {
    '3-ano-A': '3-medio-basico',
    '3-ano-B': '3-medio-basico',
    '3-ano-C': '3-medio-basico'
}
professores = {
    'angela':       {'ch': 40, 'disciplinas': ['portugues']},
    'evely':        {'ch': 40, 'disciplinas': ['matemática']},
    'vangogh':      {'ch': 40, 'disciplinas': ['arte']},
    'aristoteles':  {'ch': 40, 'disciplinas': ['biologia']},
    'platao':       {'ch': 40, 'disciplinas': ['ed-fisica']},
    'kika':         {'ch': 40, 'disciplinas': ['filosofia']},
    'apolo':        {'ch': 40, 'disciplinas': ['física']},
    'evelize':      {'ch': 40, 'disciplinas': ['geografia']},
    'benfica':      {'ch': 40, 'disciplinas': ['historia']},
    'romeu':        {'ch': 40, 'disciplinas': ['ingles']},
    'eveline':      {'ch': 40, 'disciplinas': ['quimica']},
    'rafael':       {'ch': 10, 'disciplinas': ['sociologia']},
    'afrodite':     {'ch': 20, 'disciplinas': ['redacao']}
}
minutosAula = 45
leiDo1Terco = 1/3
aulas = 5
dias = 5
for p in professores:
    p = professores[p]
    p['ch_liquida'] = int(round(p['ch'] - (p['ch'] * leiDo1Terco), 0))
    p['aulas'] = int((p['ch_liquida'] * 60) / minutosAula)
    p['aulas_sequencia'] = 3 if p['ch'] == 40 else 2 if p['ch'] >= 20 else 1
    p['aulas_usadas'] = np.zeros((aulas, dias), int)
    p['aulas_excessao'] = np.zeros((aulas, dias), int)
pd.DataFrame(professores)

,angela,evely,vangogh,aristoteles,platao,kika,apolo,evelize,benfica,romeu,eveline,rafael,afrodite
ch,40,40,40,40,40,40,40,40,40,40,40,10,20
disciplinas,[portugues],[matemática],[arte],[biologia],[ed-fisica],[filosofia],[física],[geografia],[historia],[ingles],[quimica],[sociologia],[redacao]
ch_liquida,27,27,27,27,27,27,27,27,27,27,27,7,13
aulas,36,36,36,36,36,36,36,36,36,36,36,9,17
aulas_sequencia,3,3,3,3,3,3,3,3,3,3,3,1,2
aulas_usadas,"[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0..."
aulas_excessao,"[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0...","[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0..."


In [187]:
mAulas = np.zeros((aulas, dias), int)
for turma in turmas:
    grade = grades[turmas[turma]]
    


3-ano-A {'arte': 1, 'biologia': 3, 'ed-fisica': 1, 'filosofia': 1, 'física': 3, 'geografia': 2, 'historia': 2, 'ingles': 2, 'portugues': 4, 'matematica': 40, 'quimica': 4, 'sociologia': 1, 'redacao': 1}
3-ano-B {'arte': 1, 'biologia': 3, 'ed-fisica': 1, 'filosofia': 1, 'física': 3, 'geografia': 2, 'historia': 2, 'ingles': 2, 'portugues': 4, 'matematica': 40, 'quimica': 4, 'sociologia': 1, 'redacao': 1}
3-ano-C {'arte': 1, 'biologia': 3, 'ed-fisica': 1, 'filosofia': 1, 'física': 3, 'geografia': 2, 'historia': 2, 'ingles': 2, 'portugues': 4, 'matematica': 40, 'quimica': 4, 'sociologia': 1, 'redacao': 1}


In [181]:
professores['angela']['aulas_usadas']

array([[1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [182]:
professores['angela']['aulas_excessao']

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])